In [1]:
#from tensorflow_docs.vis import embed
from tensorflow import keras
#from imutils import paths

#import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
#import imageio
import cv2
#from ipywidgets import Video, Image

import os
import json

Harold:

There are 3 paths that you have to change and complete with your own paths:
1. to load "train df"
2. "video_path" which is the address to the data_aug_video folder
3. "path_save_embeddings" folder in which you'll save the embeddings

note: here we only process the train dataset, we don't need the test dataset


Loading data_aug df

In [2]:
#1
train_df= pd.read_csv('/home/app/src/CSV/df_trailer_aug_6826_4_labels.csv')
train_df

,video_aug_name,tag
0,tt1376168_0034_aug_0.mp4,Pull
1,tt1376168_0034_aug_1.mp4,Pull
2,tt1376168_0034_aug_2.mp4,Pull
3,tt1376168_0034_aug_3.mp4,Pull
4,tt1376168_0034_aug_4.mp4,Pull
...,...,...
6821,tt6210456_0021_aug_5.mp4,Pull
6822,tt6210456_0021_aug_6.mp4,Pull
6823,tt6210456_0021_aug_9.mp4,Pull
6824,shot_tt0444850_0014.mp4,Static


building a new column with video current address

In [3]:
# Script for build new column with video current address

# path to the location of the aug videos folder 
#2
video_path = '/home/app/src/data/trailer_aug_6826/'

new_addr_list=[]

for idx_videos, video_aug in train_df.iterrows():
    new_addr = str(video_path+ video_aug.video_aug_name)
    new_addr_list.append(new_addr)

train_df['video_name'] = new_addr_list

# if you want to save .csv use this
#train_df.to_csv('/home/danield/AnyoneAI/Curso/final_project/videos_data_aug/vidaug/CSV/dataset_to_aug_LOCAL.csv', index = False)

In [4]:
train_df

,video_aug_name,tag,video_name
0,tt1376168_0034_aug_0.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt1376168_...
1,tt1376168_0034_aug_1.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt1376168_...
2,tt1376168_0034_aug_2.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt1376168_...
3,tt1376168_0034_aug_3.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt1376168_...
4,tt1376168_0034_aug_4.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt1376168_...
...,...,...,...
6821,tt6210456_0021_aug_5.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt6210456_...
6822,tt6210456_0021_aug_6.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt6210456_...
6823,tt6210456_0021_aug_9.mp4,Pull,/home/app/src/data/trailer_aug_6826/tt6210456_...
6824,shot_tt0444850_0014.mp4,Static,/home/app/src/data/trailer_aug_6826/shot_tt044...


In [5]:
# Prevent tensorflow to allocate the entire GPU
# https://www.tensorflow.org/api_docs/python/tf/config/experimental/set_memory_growth
physical_devices = tf.config.list_physical_devices("GPU")
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

2022-11-20 20:40:28.685126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 20:40:28.694380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 20:40:28.695241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Embedding script

In [6]:
# Preparing train df
print(f"Total videos for training: {len(train_df)}")
train_df.sample(10)

# Plotting df distributions
print(f'Train value counts:' '\n',train_df['tag'].value_counts(), '\n')

# Defining hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 20
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


# Loading frames and working on frame's size
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

# Feature Extractor
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()
print(feature_extractor.summary())

# Working on labels
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

# Preparing videos

def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

        print(f' {root_dir}_video {idx}/{len(video_paths)} done')

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
print(f"Frame features in train set: {train_data[0].shape}")


Total videos for training: 6826
Train value counts:
 Push      4240
Pull      2584
Static       1
Motion       1
Name: tag, dtype: int64 



2022-11-20 20:40:28.953371: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 20:40:28.953904: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 20:40:28.954789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-20 20:40:28.955579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

87924736/87910968 [==============================] - 2s 0us/step
Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________

In [ ]:
# Saving embeddings
#3.
path_save_embeddings= '/home/app/src/embeddings/mov_data_aug_6826_embeddings_20F/'

# Train
train_data_embedding_0 = np.save(path_save_embeddings + 'train_data_embedding_0.npy', train_data[0])
train_data_embedding_1 = np.save(path_save_embeddings + 'train_data_embedding_1.npy', train_data[1])
train_labels_embedding = np.save(path_save_embeddings + 'train_labels_embedding.npy', train_labels)
print(f'train data augembeddings saved OK')